In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from NPSmethods import*
import pickle as pkl
import glob
import asyncio

In [2]:
output = widgets.Output()

@output.capture(clear_output=False,wait=True)

#Defining button actions
def GoodFit(b):
    Label = 1
    
    return

def BadFit(b):
    Label = 0
    
    return

def SaveAndQuit(b):
    stop = 1
    
    return
    
#Monitors for button click
def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True)
    widget.on_click(getvalue)
    return future

#Creating ipywidget buttons
GoodButton = widgets.Button(
    description = 'Good Fit'
)
BadButton = widgets.Button(
    description = 'Bad Fit'
)
QuitButton = widgets.Button(
    description = 'Save and Quit'
)

ContinueButton = widgets.Button(
    description = 'Continue'
)

#Assigning button actions
GoodButton.on_click(GoodFit)
BadButton.on_click(BadFit)
QuitButton.on_click(SaveAndQuit)

In [3]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : True,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

path = r'C:\directory\raw_image'

resDir = r"C:\directory\results"

trapRegion = (slice(100, 200), slice(100, 200))
    # the region where the atoms located, [(xmin, xmax), (ymin, ymax)] (pixel)

noiseRegion = (slice(0, 300), slice(0, 300))
    # the region chosen for noise analysis, [(xmin, xmax), (ymin, ymax)] (pixel)

numOfImgsInEachRun = 50      # number of images for each run of analysis

rampingParameter = 0

In [4]:
#Asynchronous loop that allows for a continue button
async def StartComparing():
    #Variables and arrays used in this function
    Exp = []
    Fit = []
    npsLabels = []
    Errors = []
    Folders = []
    Label = 0
    stop = 0
    
    #Iterate over image folders in directory
    for folder in glob.iglob(path + '\\*'):
        print("LOADING NEXT IMAGE")
        clear_output(wait=True)
        
        #Attempt to run NPSmethods functions and print the fit and exp. nps (fails for non-zero ramping parameter)
        try:
            _, _, M2k_Exp, M2k_Fit, _, _ = \
            doCalibration(folder, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
        except:
            Errors = np.append(Errors, folder)
            continue
        
        #Display buttons
        display(GoodButton, BadButton, QuitButton, ContinueButton)
        
        #Pauses loop while waiting for continue button click
        x = await wait_for_change(ContinueButton)
        
        #Append imaging response functions and labels
        Exp = np.append(Exp, M2k_Exp)
        Fit = np.append(Fit, M2k_Fit)
        npsLabels = np.append(npsLabels, Label)
        Folders = np.append(Folders, folder)
        
        #If SaveAndQuit has been pressed, exit loop
        if stop == 1:
            clear_output()
            break
        
        clear_output()
    
    #Print folders with non-zero ramping parameter
    print("M2K data and labels have been saved")
    print("The following folders have non-zero ramping parameter:")
    for error in Errors:
        print(error)

    #Save data as a pickle file
    pkl.dump([npsLabels, Folders, [Exp, Fit]], open(resDir + '\\npsImgsAndLabels','wb'))

In [5]:
#Press Good Fit or Bad Fit and optionally SaveAndQuit to stop early then press Continue button to execute actions
asyncio.create_task(StartComparing())

M2K data and labels have been saved
The following folders have non-zero ramping parameter:
C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image\061426
C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image\061427
